# TD9

## Exercice 2

In [ ]:
public class Graph {
    private final int nV;
    private int nE;
    private List<Integer>[] adj;

    public Graph(int nV) {
        if (nV < 0) { throw new IllegalArgumentException("Number of vertices must be nonnegative"); }
        this.nV = nV;
        this.nE = 0;
        adj = new ArrayList[nV];
        for (int v = 0; v < nV; v++) {
            adj[v] = new ArrayList<Integer>();
        }
    }

    public int numberOfVertices() { return nV; }
    public int numberOfEdges() { return nE; }

    private void validateVertex(int v) {
        if (v < 0 || v >= nV) { throw new IndexOutOfBoundsException("vertex " + v + " is not between 0 and " + (nV - 1)); }
    }

    public void addEdge(int v, int w) {
        validateVertex(v);
        validateVertex(w);
        nE++;
        adj[v].add(w);
        adj[w].add(v);
    }

    public Iterable<Integer> adjacency(int v) {
        validateVertex(v);
        return adj[v];
    }
    
    public String toString() {
        StringBuilder s = new StringBuilder();
        String NEWLINE = System.getProperty("line.separator");
        s.append(nV + " vertices, " + nE + " edges " + NEWLINE);
        for (int v = 0; v < nV; v++) {
            s.append(v + ": ");
            for (int w : adj[v]) {
                s.append(w + " ");
            }
            s.append(NEWLINE);
        }
        return s.toString();
    }
}

In [ ]:
Graph G = new Graph(4);
G.addEdge(0, 1);
G.addEdge(0, 2);
G.addEdge(1, 3);
G.addEdge(2, 3);
System.out.println(G);
/* Doit afficher :
4 vertices, 4 edges 
0: 1 2 
1: 0 3 
2: 0 3 
3: 1 2 */

## Exercice 3

In [ ]:
public class BreadthFirstSearch {
    private static final int INFINITY = Integer.MAX_VALUE;
    private boolean[] marked; // marked[v] = is there an s-v path
    private int[] distTo;     // distTo[v] = number of edges on shortest s-v path

    // Computes the shortest path between the source vertex s
    // and every other vertex in the graph
    public BreadthFirstSearch(Graph G, int s) {
        marked = new boolean[G.numberOfVertices()];
        distTo = new int[G.numberOfVertices()];
        bfs(G, s);
    }

    public boolean hasPathTo(int v) { return marked[v]; }
    public int distanceTo(int v) { return distTo[v]; }
    
    private void bfs(Graph G, int s) {
        Deque<Integer> queue = new ArrayDeque<Integer>();
        for (int v = 0; v < G.numberOfVertices(); v++) {
          distTo[v] = INFINITY;
        }
        distTo[s] = 0;
        marked[s] = true;
        queue.addLast(s);
        while (!queue.isEmpty()) {
            int v = queue.removeFirst();
            for (int w : G.adjacency(v)) {
                if (!marked[w]) {
                    distTo[w] = distTo[v] + 1;
                    marked[w] = true;
                    queue.addLast(w);
                }
            }
        }
    }
}

In [ ]:
Graph labyrinth = new Graph(6);
labyrinth.addEdge(0, 1);
labyrinth.addEdge(1, 2);
labyrinth.addEdge(2, 3);
labyrinth.addEdge(1, 4);
labyrinth.addEdge(4, 5);
int start = 0;
int exit = 5;

BreadthFirstSearch bfs = new BreadthFirstSearch(labyrinth, start);
if (bfs.hasPathTo(exit)) { System.out.println("We need " + bfs.distanceTo(exit) + " steps to reach the exit"); }
else { System.out.println("No hope"); }
/* Doit afficher :
We need 3 steps to reach the exit */

## Exercice 4

In [ ]:
public class BreadthFirstSearch {
    private static final int INFINITY = Integer.MAX_VALUE;
    private boolean[] marked; // marked[v] = is there an s-v path
    private int[] distTo;     // distTo[v] = number of edges on shortest s-v path
    private int[] edgeTo;     // edgeTo[v] = previous edge on shortest s-v path

    // Computes the shortest path between the source vertex s
    // and every other vertex in the graph
    public BreadthFirstSearch(Graph G, int s) {
        marked = new boolean[G.numberOfVertices()];
        distTo = new int[G.numberOfVertices()];
        edgeTo = new int[G.numberOfVertices()];
        bfs(G, s);
    }

    public boolean hasPathTo(int v) { return marked[v]; }
    public int distanceTo(int v) { return distTo[v]; }
    
    private void bfs(Graph G, int s) {
        Deque<Integer> queue = new ArrayDeque<Integer>();
        for (int v = 0; v < G.numberOfVertices(); v++) {
          distTo[v] = INFINITY;
        }
        distTo[s] = 0;
        marked[s] = true;
        queue.addLast(s);
        while (!queue.isEmpty()) {
            int v = queue.removeFirst();
            for (int w : G.adjacency(v)) {
                if (!marked[w]) {
                    edgeTo[w] = v;
                    distTo[w] = distTo[v] + 1;
                    marked[w] = true;
                    queue.addLast(w);
                }
            }
        }
    }
    
    public void printPathTo(int destination) {
        if (distanceTo(destination) != 0) printPathTo(edgeTo[destination]);
        System.out.println(destination);
    }
}

In [ ]:
Graph labyrinth = new Graph(6);
labyrinth.addEdge(0, 1);
labyrinth.addEdge(1, 2);
labyrinth.addEdge(2, 3);
labyrinth.addEdge(1, 4);
labyrinth.addEdge(4, 5);
int start = 0;
int exit = 5;

BreadthFirstSearch bfs = new BreadthFirstSearch(labyrinth, start);
if (bfs.hasPathTo(exit)) {
    System.out.println("We need " + bfs.distanceTo(exit) + " steps to reach the exit");
    bfs.printPathTo(exit);
}
else { System.out.println("No hope"); }
/* Doit afficher :
We need 3 steps to reach the exit
0
1
4
5 */

## Exercice 5

In [ ]:
public class ConnectedComponents {
    private boolean[] marked; // marked[v] = has vertex v been marked?
    private int count;        // number of connected components
    
    // Computes the number of connected components
    public ConnectedComponents(Graph G) {
        marked = new boolean[G.numberOfVertices()];
        for (int v = 0; v < G.numberOfVertices(); v++) {
            if (!marked[v]) {
                dfs(G, v);
                count++;
            }
        }
    }
    
    // depth-first search
    private void dfs(Graph G, int v) {
        marked[v] = true;
        for (int w : G.adjacency(v)) {
            if (!marked[w]) {
                dfs(G, w);
            }
        }
    }
    
    public int count() { return count; }
}

In [ ]:
Graph G = new Graph(6);
G.addEdge(0, 1);
G.addEdge(0, 2);
G.addEdge(1, 2);
G.addEdge(3, 4);
G.addEdge(5, 5);
System.out.println(G);
ConnectedComponents cc = new ConnectedComponents(G);
System.out.println("connected components count: " + cc.count());

/* Doit afficher :
6 vertices, 5 edges 
0: 1 2 
1: 0 2 
2: 0 1 
3: 4 
4: 3 
5: 5 5 

connected components count: 3 */

## Exercice 6

In [ ]:
public class Digraph {
    private final int V;
    private int E;
    private List<Integer>[] adj;

    @SuppressWarnings("unchecked")
    public Digraph(int V) {
        if (V < 0)
            throw new IllegalArgumentException(
                    "Number of vertices must be nonnegative");
        this.V = V;
        this.E = 0;
        adj = new ArrayList[V];
        for (int v = 0; v < V; v++) {
            adj[v] = new ArrayList<Integer>();
        }
    }

    public int numberOfVertices() {
        return V;
    }

    public int numberOfEdges() {
        return E;
    }

    private void validateVertex(int v) {
        if (v < 0 || v >= V)
            throw new IndexOutOfBoundsException("vertex " + v
                    + " is not between 0 and " + (V - 1));
    }

    public void addEdge(int v, int w) {
        validateVertex(v);
        validateVertex(w);
        E++;
        adj[v].add(w);
    }

    public Iterable<Integer> adjacency(int v) {
        validateVertex(v);
        return adj[v];
    }
    
    public String toString() {
        StringBuilder s = new StringBuilder();
        String NEWLINE = System.getProperty("line.separator");
        s.append(V + " vertices, " + E + " edges " + NEWLINE);
        for (int v = 0; v < V; v++) {
            s.append(String.format("%d: ", v));
            for (int w : adj[v]) {
                s.append(String.format("%d ", w));
            }
            s.append(NEWLINE);
        }
        return s.toString();
    }
}

In [ ]:
public class TopologicalOrder {
    private boolean[] marked; // marked[v] = has vertex v been marked?
    private Deque<Integer> stack; // topological order

    public TopologicalOrder(Digraph G) {
      stack = new ArrayDeque<Integer>();
      marked = new boolean[G.numberOfVertices()];
      for (int v = 0; v < G.numberOfVertices(); v++) {
        if (!marked[v]) {
            dfs(G, v);
        }
      }
    }
    
    // depth-first search
    private void dfs(Digraph G, int v) {
        marked[v] = true;
        for (int w : G.adjacency(v)) {
            if (!marked[w]) {
                dfs(G, w);
            }
        }
        stack.addFirst(v);
    }
    
    public Iterable<Integer> order() { return stack; }
    
    public String toString() {
        StringBuilder s = new StringBuilder();
        for (int v : order()) {
            s.append(v + " ");
        }
        return s.toString();
    }
}

In [ ]:
Digraph G = new Digraph(6);
G.addEdge(0, 1);
G.addEdge(0, 2);
G.addEdge(1, 2);
G.addEdge(3, 1);
G.addEdge(3, 4);
G.addEdge(4, 5);
TopologicalOrder order = new TopologicalOrder(G);
System.out.println(G);
System.out.println("order: " + order);
/* Doit afficher :
6 vertices, 6 edges 
0: 1 2 
1: 2 
2: 
3: 1 4 
4: 5 
5: 

order: 3 4 5 0 1 2 */

In [ ]:
G = new Digraph(8);
G.addEdge(0, 2);
G.addEdge(1, 2);
G.addEdge(2, 3);
G.addEdge(2, 3);
G.addEdge(2, 5);
G.addEdge(3, 4);
G.addEdge(3, 5);
G.addEdge(5, 4);
G.addEdge(4, 6);
G.addEdge(5, 6);
G.addEdge(5, 7);
G.addEdge(6, 7);
order = new TopologicalOrder(G);
System.out.println(G);
System.out.println("order: " + order);
/* Doit afficher :
8 vertices, 12 edges 
0: 2 
1: 2 
2: 3 3 5 
3: 4 5 
4: 6 
5: 4 6 7 
6: 7 
7: 

order: 1 0 2 3 5 4 6 7 */

## Exercice 7

In [ ]:
public class NumberOfPaths {
    int[] count;
    int nbPaths;
    
    public NumberOfPaths(Digraph G, int first, int second) {
        TopologicalOrder topo = new TopologicalOrder(G);
        count = new int[G.numberOfVertices()];
        int[] order = new int[G.numberOfVertices()];
        int indexOfFirst = 0, indexOfSecond = 0;
        int i = 0;
        for (int v : topo.order()) {
            order[i] = v;
            if (v == first) indexOfFirst = i;
            if (v == second) indexOfSecond = i;
            ++i;
        }
        count[first] = 1;
        for (i = indexOfFirst; i < indexOfSecond; i++) {
            for (int neighbor : G.adjacency(order[i])) {
                count[neighbor] += count[order[i]];
            }
        }
        nbPaths = count[second];
    }
    
    public int numberOfPaths() {
        return nbPaths;
    }
}

In [ ]:
Digraph G = new Digraph(8);
G.addEdge(0, 2);
G.addEdge(1, 2);
G.addEdge(2, 3);
G.addEdge(2, 3);
G.addEdge(2, 5);
G.addEdge(3, 4);
G.addEdge(3, 5);
G.addEdge(5, 4);
G.addEdge(4, 6);
G.addEdge(5, 6);
G.addEdge(5, 7);
G.addEdge(6, 7);
System.out.println(G);
NumberOfPaths p = new NumberOfPaths(G, 2, 7);
System.out.println("number of paths between 2 and 7: " + p.numberOfPaths());
/* Doit afficher :
8 vertices, 12 edges 
0: 2 
1: 2 
2: 3 3 5 
3: 4 5 
4: 6 
5: 4 6 7 
6: 7 
7: 

number of paths between 2 and 7: 11 */